In [18]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set()
%matplotlib inline


In [19]:
import os
os.chdir('C:\\Users\\dwagn\\Downloads\\Project\\Project')

In [20]:
count_method = 'n' # 'c' or 'n' # n = n tokens, c = distinct token (term) count
tf_method = 'sum' # sum, max, log, double_norm, raw, binary
tf_norm_k = .5 # only used for double_norm
idf_method = 'standard' # standard, max, smooth
gradient_cmap = 'YlGnBu' # YlGn, GnBu, YlGnBu; For tables; see https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html 

In [21]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
SENTS = OHCO[:4]
PARAS = OHCO[:3]
CHAPS = OHCO[:2]
BOOKS = OHCO[:1]

In [22]:
LIB = pd.read_csv(data_dir + "LIB.csv").set_index(BOOKS)
TOKEN = pd.read_csv(data_dir + 'TOKEN.csv').set_index(OHCO)
VOCAB = pd.read_csv(data_dir + 'VOCAB.csv').set_index('term_id')

In [23]:
VOCAB = VOCAB[~VOCAB.term_str.isna()]
TOKEN = TOKEN[~TOKEN.term_str.isna()]

In [24]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)
VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).pos.count().unstack().idxmax(1)

In [25]:
if 'term_rank' not in VOCAB.columns:
    VOCAB = VOCAB.sort_values('n', ascending=False).reset_index()
    VOCAB.index.name = 'term_rank'
    VOCAB = VOCAB.reset_index()
    VOCAB = VOCAB.set_index('term_id')
    VOCAB['term_rank'] = VOCAB['term_rank'] + 1

In [26]:
new_rank = VOCAB.n.value_counts()\
    .sort_index(ascending=False).reset_index().reset_index()\
    .rename(columns={'level_0':'term_rank2', 'index':'n', 'n':'nn'})\
    .set_index('n')

In [27]:
VOCAB['term_rank2'] = VOCAB.n.map(new_rank.term_rank2) + 1
VOCAB['p'] = VOCAB.n / VOCAB.shape[0]

In [28]:
VOCAB['zipf_k'] = VOCAB.n * VOCAB.term_rank
VOCAB['zipf_k2'] = VOCAB.n * VOCAB.term_rank2
VOCAB['zipf_k3'] = VOCAB.p * VOCAB.term_rank2

In [29]:
VOCAB2 = VOCAB.copy()

In [30]:
def TFIDF_Matrix(df,bag,count_type,tf_type,idf_type):

    BOW = TOKEN.groupby(bag+['term_id']).term_id.count()\
    .to_frame().rename(columns={'term_id':'n'})

    BOW['c'] = BOW.n.astype('bool').astype('int')
    
    DTCM = BOW[count_method].unstack().fillna(0).astype('int')

    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()

    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()

    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)
    
    elif tf_method == 'raw':
        TF = DTCM.T

    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
        TF = tf_norm_k + (1 - tf_norm_k) * TF[TF > 0] # EXPLAIN; may defeat purpose of norming

    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    
    TF = TF.T

    DF = DTCM[DTCM > 0].count()

    N = DTCM.shape[0]

    if idf_method == 'standard':
        IDF = np.log10(N / DF)

    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 

    elif idf_method == 'smooth':
        IDF = np.log10((1 + N) / (1 + DF)) + 1

    TFIDF = TF * IDF

    VOCAB['df'] = DF
    VOCAB['idf'] = IDF

    BOW['tf'] = TF.stack()
    BOW['tfidf'] = TFIDF.stack()

    VOCAB['tfidf_sum'] = TFIDF.sum()

    VOCAB.sort_values('tfidf_sum', ascending=False).head(20).style.background_gradient\
        (cmap=gradient_cmap, high=1)

    VOCAB[['term_rank','term_str','pos_max','tfidf_sum']]\
        .sort_values('tfidf_sum', ascending=False).head(50)\
        .style.background_gradient(cmap=gradient_cmap, high=1)

    VOCAB.loc[VOCAB.pos_max != 'NNP', ['term_rank','term_str','pos_max','tfidf_sum']]\
        .sort_values('tfidf_sum', ascending=False)\
        .head(50).style.background_gradient(cmap=gradient_cmap, high=1)
    
    BOW = BOW.join(VOCAB[['term_str','pos_max']], on='term_id')

    BOW.sort_values('tfidf', ascending=False).head(20)\
        .style.background_gradient(cmap=gradient_cmap, high=1)

    return TFIDF

In [31]:
TFIDF = TFIDF_Matrix(TOKEN,BOOKS,count_method,tf_method,idf_method)

In [32]:
VOCAB.sort_values('tfidf_sum', ascending=False).head(20).style.background_gradient(cmap=gradient_cmap, high=1)

,term_rank,term_str,n,num,stop,p_stem,pos_max,term_rank2,p,zipf_k,zipf_k2,zipf_k3,df,idf,tfidf_sum
term_id,,,,,,,,,,,,,,,
3976,67,fanny,816,0,0,fanni,NNP,67,0.068675,54672,54672,4.601246,1,0.477121,0.002436
3440,90,elizabeth,585,0,0,elizabeth,NNP,89,0.049234,52650,52065,4.381838,1,0.477121,0.002294
2402,102,crawford,493,0,0,crawford,NNP,101,0.041491,50286,49793,4.190624,1,0.477121,0.001472
2521,126,darcy,365,0,0,darci,NNP,123,0.030719,45990,44895,3.778404,1,0.477121,0.001431
10621,228,tilney,196,0,0,tilney,NNP,195,0.016496,44688,38220,3.216630,1,0.477121,0.001207
1043,156,bennet,293,0,0,bennet,NNP,148,0.024659,45708,43364,3.649554,1,0.477121,0.001149
1524,95,catherine,538,0,0,catherin,NNP,94,0.045279,51110,50572,4.256186,2,0.176091,0.001132
3385,127,edmund,365,0,0,edmund,NNP,123,0.030719,46355,44895,3.778404,1,0.477121,0.001090
5865,171,jane,258,0,0,jane,NNP,159,0.021714,44118,41022,3.452449,1,0.477121,0.001012


In [33]:
TFIDF_Matrix(TOKEN,CHAPS,count_method,tf_method,idf_method)

term_id              1      2      3      4      5         6      7      \
book_id chap_num                                                          
121     31        0.000000    0.0    0.0    0.0    0.0  0.013108    0.0   
        32        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
        33        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
        34        0.001249    0.0    0.0    0.0    0.0  0.000000    0.0   
        35        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
...                    ...    ...    ...    ...    ...       ...    ...   
42671   57        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
        58        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
        59        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
        60        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   
        61        0.000000    0.0    0.0    0.0    0.0  0.000000    0.0   

term_id           8      9      10     ...     11873  11874  11875     11876  \
book_id chap_num                       ...                                     
121     31          0.0    0.0    0.0  ...  0.000000    0.0    0.0  0.000000   
        32          0.0    0.0    0.0  ...  0.000000    0.0    0.0  0.000679   
        33          0.0    0.0    0.0  ...  0.000136    0.0    0.0  0.000000   
        34          0.0    0.0    0.0  ...  0.000173    0.0    0.0  0.000000   
        35          0.0    0.0    0.0  ...  0.000000    0.0    0.0  0.000000   
...                 ...    ...    ...  ...       ...    ...    ...       ...   
42671   57          0.0    0.0    0.0  ...  0.000540    0.0    0.0  0.000000   
        58          0.0    0.0    0.0  ...  0.000245    0.0    0.0  0.000000   
        59          0.0    0.0    0.0  ...  0.000361    0.0    0.0  0.000000   
        60          0.0    0.0    0.0  ...  0.000564    0.0    0.0  0.000000   
        61          0.0    0.0    0.0  ...  0.000000    0.0    0.0  0.000741   

term_id           11877  11878  11879  11880  11881  11882  
book_id chap_num                                            
121     31          0.0    0.0    0.0    0.0    0.0    0.0  
        32          0.0    0.0    0.0    0.0    0.0    0.0  
        33          0.0    0.0    0.0    0.0    0.0    0.0  
        34          0.0    0.0    0.0    0.0    0.0    0.0  
        35          0.0    0.0    0.0    0.0    0.0    0.0  
...                 ...    ...    ...    ...    ...    ...  
42671   57          0.0    0.0    0.0    0.0    0.0    0.0  
        58          0.0    0.0    0.0    0.0    0.0    0.0  
        59          0.0    0.0    0.0    0.0    0.0    0.0  
        60          0.0    0.0    0.0    0.0    0.0    0.0  
        61          0.0    0.0    0.0    0.0    0.0    0.0  

[141 rows x 11882 columns]

In [34]:
VOCAB.to_csv('VOCAB2.csv')
TOKEN.to_csv('TOKEN2.csv')
TFIDF.to_csv('TFIDF.csv')